In [2]:
!pip install searchtweets

In [4]:
!pip install textblob

     |████████████████████████████████| 636 kB 1.7 MB/s            
     |████████████████████████████████| 1.5 MB 51.6 MB/s            
     |████████████████████████████████| 763 kB 52.8 MB/s            


In [6]:
!pip install afinn

     |████████████████████████████████| 52 kB 1.2 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53448 sha256=baff22eac21fbc748b91581bad2edd07f4d28d87aa689144b8f5d067e225d87f
  Stored in directory: /home/jovyan/.cache/pip/wheels/79/91/ee/8374d9bc8c6c0896a2db75afdfd63d43653902407a0e76cd94
Successfully built afinn


In [31]:
import pandas as pd
import numpy as np
import requests
import searchtweets
from textblob import TextBlob
from afinn import Afinn
from datetime import *
import re

In [2]:
df = pd.read_csv("~/work/projetpy/fichiers_csv/tweets_2021-3-1_to_2021-4-1.csv")
#afinn = Afinn()
#afinn_scores = [afinn.score(text) for text in df.text]
#df['afinn'] = afinn_scores
#df[['afinn', 'text']].head()

In [10]:
b = TextBlob("Je suis en colère")
b.detect_language()
b.sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [11]:
df.head()

,id,date,screenName,text,type
0,1377376779178967041,2021-03-31 21:46:29,ratonlave57,Pourquoi ne donne t’on pas de l’Aspegic pédiat...,tweet
1,1377339517170933768,2021-03-31 19:18:25,MathildeSlv,@mvrinesv Le syndrome grippal du Pfizer existe...,tweet
2,1377337776819990528,2021-03-31 19:11:30,EBrussat,@montagnethiers #Orleat 210 personnes #vaccine...,tweet
3,1377336854379331584,2021-03-31 19:07:50,garrec_pierre,"En tout cas vendredi, c’est le vaccin astrazen...",tweet
4,1377325612134699012,2021-03-31 18:23:10,FPasquisDumont,1700 centres Pfizer et Moderna...\nPlus de 70 ...,tweet


# Création des métadonnées

Nombre de hashtags

In [32]:
def decompte_hashtags(text):
    exp_match = re.compile("([#]\w+)")
    res = exp_match.findall(text)
    return len(res)

df["nbre_hashtags"] = df["text"].apply(lambda x : decompte_hashtags(x))

Nombre d'url

In [40]:
def decompte_url(text):
    exp_match = re.compile("([https//:]\w+[.]\w+)")
    res = exp_match.findall(text)
    return len(res)

df["nbre_url"] = df["text"].apply(lambda x : decompte_url(x))

Nombre de points d'exclamtion

In [41]:
def decompte_exclamation(text):
    exp_match = re.compile("(\w?\s?[!])")
    res = exp_match.findall(text)
    return len(res)

df["nbre_exclamation"] = df["text"].apply(lambda x : decompte_exclamation(x))
df

,id,date,screenName,text,type,nbre_hashtags,nbre_url,nbre_exclamation
0,1377376779178967041,2021-03-31 21:46:29,ratonlave57,Pourquoi ne donne t’on pas de l’Aspegic pédiat...,tweet,0,0,0
1,1377339517170933768,2021-03-31 19:18:25,MathildeSlv,@mvrinesv Le syndrome grippal du Pfizer existe...,tweet,0,0,0
2,1377337776819990528,2021-03-31 19:11:30,EBrussat,@montagnethiers #Orleat 210 personnes #vaccine...,tweet,5,0,0
3,1377336854379331584,2021-03-31 19:07:50,garrec_pierre,"En tout cas vendredi, c’est le vaccin astrazen...",tweet,0,0,0
4,1377325612134699012,2021-03-31 18:23:10,FPasquisDumont,1700 centres Pfizer et Moderna...\nPlus de 70 ...,tweet,0,0,0
...,...,...,...,...,...,...,...,...
95,1375959954906054659,2021-03-27 23:56:32,Indoartisk5,@MinisteredelaS1 vaccin première injection d'A...,tweet,0,1,0
96,1375917310288990215,2021-03-27 21:07:04,Nadinemarie762,ÇA c'est une bonne nouvelle !!!!\nLa SEULE bon...,quote,0,3,0
97,1375916974807650306,2021-03-27 21:05:44,Nadinemarie762,ÇA c'est une bonne nouvelle !!!!\nLa SEULE bon...,tweet,0,1,0
98,1375916691499155460,2021-03-27 21:04:37,Nadinemarie762,ÇA c'est une bonne nouvelle !!!!\nLa SEULE bon...,tweet,0,1,0


In [51]:
def decompte_suspension(text):
    exp_match = re.compile("(\w?\s?[.]{2,5})")
    res = exp_match.findall(text)
    return 0 if len(res)== 0 else 1
df["presence_suspension"] = df["text"].apply(lambda x : decompte_suspension(x))
df

,id,date,screenName,text,type,nbre_hashtags,nbre_url,nbre_exclamation,presence_suspension
0,1377376779178967041,2021-03-31 21:46:29,ratonlave57,Pourquoi ne donne t’on pas de l’Aspegic pédiat...,tweet,0,0,0,0
1,1377339517170933768,2021-03-31 19:18:25,MathildeSlv,@mvrinesv Le syndrome grippal du Pfizer existe...,tweet,0,0,0,0
2,1377337776819990528,2021-03-31 19:11:30,EBrussat,@montagnethiers #Orleat 210 personnes #vaccine...,tweet,5,0,0,0
3,1377336854379331584,2021-03-31 19:07:50,garrec_pierre,"En tout cas vendredi, c’est le vaccin astrazen...",tweet,0,0,0,0
4,1377325612134699012,2021-03-31 18:23:10,FPasquisDumont,1700 centres Pfizer et Moderna...\nPlus de 70 ...,tweet,0,0,0,1
...,...,...,...,...,...,...,...,...,...
95,1375959954906054659,2021-03-27 23:56:32,Indoartisk5,@MinisteredelaS1 vaccin première injection d'A...,tweet,0,1,0,0
96,1375917310288990215,2021-03-27 21:07:04,Nadinemarie762,ÇA c'est une bonne nouvelle !!!!\nLa SEULE bon...,quote,0,3,0,0
97,1375916974807650306,2021-03-27 21:05:44,Nadinemarie762,ÇA c'est une bonne nouvelle !!!!\nLa SEULE bon...,tweet,0,1,0,0
98,1375916691499155460,2021-03-27 21:04:37,Nadinemarie762,ÇA c'est une bonne nouvelle !!!!\nLa SEULE bon...,tweet,0,1,0,0
